In [ ]:
import pandas as pd 
pd_data=pd.read_excel("/Users/lee/Documents/work/干预助手/dataset/原始数据/问题集.xls")
pd_data.info()

In [ ]:
pd_data['问题'].tolist()

In [13]:
import requests
import json

def send_prediction_request( query,url="http://localhost:8082/api/predict"):
    """
    发送预测请求到指定的API端点。

    参数:
    url (str): API的URL地址。
    query (str): 要发送的查询字符串。

    返回:
    dict: API的响应内容。
    """
    # 定义请求头
    headers = {
        "Content-Type": "application/json"
    }

    # 定义请求体
    data = {
        "data": [query]
    }

    try:
        # 发送POST请求
        response = requests.post(url, headers=headers, data=json.dumps(data))
        
        # 检查响应状态码
        if response.status_code == 200:
            return response.json()
        else:
            response.raise_for_status()  # 抛出HTTPError异常
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP错误: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"连接错误: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"超时错误: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"请求错误: {req_err}")

    return None

In [ ]:
query = "介绍下康复师"
response = send_prediction_request(query)

if response:
    print(f"响应: {json.loads(response['data'][0])}")
else:
    print("请求失败")

In [ ]:
json.loads(response['data'][0])['response']

In [63]:
import numpy as np 
np.random.seed(10)
pd_data_sample=pd_data.sample(n=10)
pd_data_sample['result']=pd_data_sample['问题'].apply(lambda x:send_prediction_request(x))

In [ ]:
pd_data_sample['result']

In [65]:
pd_data_sample['model_response']=pd_data_sample['result'].apply(lambda x:json.loads(x['data'][0])['response'])
pd_data_sample['prompt']=pd_data_sample['result'].apply(lambda x:json.loads(x['data'][0])['prompt'])

In [ ]:
pd_data_sample

In [67]:
pd_data_sample.to_csv("test_result.csv",encoding='utf-8-sig')

In [ ]:
test_pd=pd.read_csv('test_result.csv')
test_pd

In [73]:
from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "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"
)

completion = client.chat.completions.create(
  model="deepseek-ai/deepseek-r1",
  messages=[{"role":"user","content":"基于以下已知信息，简洁和专业的来回答用户的问题。用简体中文回答。\n\n已知内容:\n[1]      \"饮料等。气体：比如冷热的风、蒸汽等。\n输入冷热的方法：空间上的对比：\n同时呈现冷热两种状态的物品，比如一个冷包子和一个热包子。让孩子自己摸一下，感受冷热的不同。比如，他摸到冷包子时，输入“冷冷的包子”；摸到热包子时，输入“热热的包子”。\n初期可以使用极端的样本，比如从冰箱拿出的冷包子或很烫的热水，给孩子留下深刻印象。\n时间上的对比：让孩子体验物品从冷到热或从热到冷的变化。比如，把冷包子放进微波炉加热，输入“包子热了”；或者把热包子放凉，输入“包子冷了”。\n意义样本的目的是让孩子体验到“冷的可以，热的不行”或“热的可以，冷的不行”的结果差异。比如：冷包子可以吃，但热包子太烫，需要等一会儿。热水可以喝，但冷水太冰，需要兑热水。\n我们可以通过日常生活中的场景和物品来输入和测试。比如：泡奶，用热水泡奶可以冲开，而冷水会导致结块，口感也不好。热水可以泡开面条，而冷水则会让面条结成一坨，不好吃。用热水浇花会把花烫死，这是一个后期可以使用的意义样本。洗完澡后用吹风机吹头发，可以输入“冷风”和“热风”。夏天开空调时，输入“冷风”或“这个房间好热”。车停在室外晒一中午后，打开车门会感受到热浪，输入“好热呀”。座椅被晒得很烫，坐上去时可以输入“座椅好热”。冷热的包子、面条等。可以通过选择疑问句输入，比如“你想吃冷的包子还是热的包子？”。\n家长：老师，我问个问题。在生活中有很多近义词，比如“凉”“冰”。我们在输入时，是可以一起说，还是要分开教学，像同义词那样教学？\n源爸：一般来说，我们的目标是让孩子听懂“冷”和“热”，但\"\n[2]      \"束缚，他们必须了解关键词，比如“冷”或“热”。打个比方，当孩子满头大汗、气喘吁吁、口渴时，你给他一杯很烫的水。他说要喝水，但你给的是烫水，他当然喝不下去。这时，如果你是孩子，你会想到什么？你一定会想要一杯冷水，然后你慢慢领悟，恍然大悟，明白自己需要冷水。这次给你冷水，你会一下子接受。相反，比如他洗澡时，你打开水龙头或圆形喷头，喜欢用流动的水来复习，对吧？\n我记得以前北京有一位母亲，她曾问我一个问题。因为北京人早上习惯吃热气腾腾的包子，而晚餐时包子的温度则是从冰箱里取出的。她问孩子，是喜欢冷的包子还是热的包子。通过这种方式，母亲明白了孩子的偏好，即喜欢温暖的包子。这种通过选择疑问句，或是像我之前提到的喝水与不同水温的案例，都能实现高效的输入。\n如果进行测试，也可以通过指令来实现，比如像一般的饮水机，通常有热水龙头和冷水龙头。我们可以让孩子去开水龙头，选择热水或冷水，或者将一杯热水放在那里，让他去拿冷水。再比如，家中的水龙头往左转是热水，往右转是冷水，看看他是否能根据指令正确操作。\n这里需要注意一些细节，即在我们下指令时，比如让孩子去接冷水或拿杯冷水，我们一定要有后续的引导。我们一直强调这个问题，给孩子下的任何指令都必须明确其目的和时机，不能含糊其辞。为什么要这样做？什么时候需要这样做？这些都是我们需要明确的问题。\n还有，就是那个什么时候拿热水壶泡茶拿着吃？那么如果孩子答错了，因为孩子本身没有生活体验，比如冷水泡茶，那么你可以做给他看，用冷水会是什么结果？我们要教孩子一个概念，一定要把这个概念融入到孩子\"\n[3]      \"家长：教冷热具体怎么操作？您刚举的包子例子没太明白，嗯。\n源爸：比如他平时吃热包子，有天你故意给冷藏过的。他一咬皱眉就说“冰包子！冷！”，这种反差体验比光看卡片深刻多了。就像孩子摔疼一次就永远记得“疼”这个词，对吧？\n家长：样本设计要和生活关联吗？比如用他怕的辣来教？\n源爸：嗯，对！要是孩子被辣哭过，你指着眼药水说“这个辣，不能碰”，他马上懂。形容词教学关键是要有强烈感官刺激，最好能调动情绪记忆，嗯。\n家长：那后期需要增加复杂度吗？比如组合不同形容词？\n源爸：对，比如“拿大的红积木”或者“喝温的牛奶”。但前期先确保单个词掌握，后面再慢慢叠加。\n家长：现在教孩子冷热概念有什么具体方法？\n源爸：哈哈哈，是吧？比如吃包子时拿冷地让他用微波炉加热，过一两分钟摸摸看包子热了可以吃。洗澡时偶尔调冷水制造反差，比如突然用冷水浇一下说“好冷啊”，这种强烈对比效果特别好，但别长期用，否则孩子会崩溃的。\n家长：怎么测试孩子是否真理解冷热？\n源爸：摆冷热两杯水，说“拿热水”看他反应。测试前先带他摸杯子感受温度差异，拿对了要立马给奖励。比如用冷水泡奶茶根本泡不开，热水才能泡，这种生活场景最直接。\n家长：冷热和其他温度词怎么区分？\n源爸：北方习惯叫凉水的，等孩子掌握基础后把凉和冷当同义词教。你看热水壶会冒气，冷饮瓶子挂水珠，这些特征多展示。就像进空调房突然哆嗦，这时候说“好冷穿外套”特直观。\n家长：温度变化怎么教更直观？\n源爸：空调房就是现成教材啊！大太阳下进商场冷得打哆嗦，立刻输入“冷”。家里留个没开空调的卧室，突然\"\n[4]      \"你可以在不同的情境中输入这些词，比如天气很热时，你可以说“这里好凉快”，或者在夏天的小溪边，你可以说“这水好凉快”。\n家长：那怎么让他理解“冷”和“热”呢？\n源爸：你可以通过制造反差来帮助他理解。比如，平时洗澡或洗脚时用热水，突然换成冷水，他会感觉到不同，这时你可以输入“水太冷了”。这种反差的体验可以帮助他更好地理解“冷”和“热”。不过，这种反差体验要适度，不能让他一直处于“水深火热”中，否则他会受不了。\n家长：那怎么测试他是否理解了“冷”和“热”呢？\n源爸：你可以通过指令来测试。比如，让他拿一杯冷水或热水，如果他能够正确执行，说明他理解了。另外，水龙头也是一个很好的测试工具。通常，水龙头往左是热水，往右是冷水。如果他能够正确打开水龙头并接出热水或冷水，也说明他理解了“冷”和“热”。\n家长：那如果他通过视觉无法判断冷热，怎么办？\n源爸：你可以先辅助他摸一下，比如让他摸一下冷水杯，然后说“拿热水”。如果他能够按照指令拿对，说明他理解了。这种测试的前提是他之前多次使用过水龙头或接触过冷热水。\n在输入“冷”和“热”时，要注意孩子的反应。比如，如果孩子喝了一口很冰的水，你可以观察到他的反应，然后输入“这水太冷了，我给你换一些”。至于“热”的输入，取决于孩子的能力。如果你觉得他能理解，那就没问题。但要注意，不要在他感受到冷的同时输入“热”，这样会让他混淆。\n另外，输入“冷”和“热”时，要确保孩子确实感受到了冷或热。我之前有个案例，孩子对冷热不敏感，妈妈为了教他，把粥冻成冰碴子给他喝，结果孩子喝完后毫无反应。\"\n[5]      \"孩子对红绿灯特别关注，可以在没有干扰的环境中输入。\n家长：我家孩子喜欢看山上的塔灯，灯会变颜色。她看到红色时会说“红色”，看到黄色时会说“黄色”，这种输入可以吗？\n源爸：如果孩子能关注到，并且环境没有太多干扰，这种输入是有效的。\n家长：有一次我让孩子拿鞋子，他第一次拿了灰色的，我不喜欢，让他拿紫色的，他成功了。后来他爸爸也用这个方法，但孩子不理他。这是挑人，还是方法用多了？\n源爸：孩子可能更听你的话，因为你和他相处更多。另外，好方法不能频繁使用，否则会失效。就像大招不能一直放，需要冷却时间。\n家长：明白了，谢谢老师！\n源爸：教冷热时，如果孩子对冷热无感，可以通过生活中的变化让他体验。比如，平时洗澡用热水，突然用冷水刺激一下，孩子可能会立刻学会“冷”。但这种方法不能频繁使用，否则会影响孩子的正常生活。\n家长：我试过洗澡时用冷水刺激，用了3次后，孩子就不喜欢洗澡了。\n源爸：绝招用多了就不再是绝招了。\n源爸：我们来看几个关于形容词的视频。\n视频1（大小测试）：\n妈妈让孩子“拿小碗过来”，孩子拿对了。但后面妈妈说“靠近一点”，孩子听不懂。测试时，指令要简洁明确，避免多余的语言干扰。\n视频2（大小测试）：\n妈妈让孩子“拿大盘子过来”，孩子一开始拿小盘子，妈妈重复指令后，孩子才拿大盘子。长期这样，孩子会依赖大人的反应来判断对错，而不是根据指令本身。\n视频3（大小测试）：\n妈妈让孩子“拿小的西瓜放茶几上”，但孩子拿不动大西瓜，只能拿小的。这种测试设置有问题，孩子拿小的西瓜并不是因为听懂了指令，而是因为大西瓜拿不动\"\n[6]      \"事情或者一个点，那么现在你想一想，我希望你前后把因果加上去。我举个最简单例子来说，原来是哎，你去倒杯水，现在是妈妈口渴了，你去倒杯水，妈妈喝。就是你要用最简单的语言把前因后果描述一下。\n这个东西就叫因果小串联。说穿了很简单，无非就是以后你跟孩子说的每一句话，你先停一下，你想想有没有前后的东西可以加进去的？那么这个东西就是日积月累，日积月累以后，他一听到口渴，他马上就联想到下一步喝水。那么我们口渴，这个圆珠笔就变成了毛笔了，它能延伸下去喝水吗？那么很多的，比如说肚子饿了吃饭，热冷开空调或者脱衣服，那么这里面注意就是我们在输入因果小串联的时候注意几个原则，第一，你这一因一果他必须要听得懂。\n第二，不同的原因会有不同样的原因，会有不同的结果，不同的原因会导致同一个结果啊。我说起来有点绕口，就打个比方说，我感觉冷了是一个原因，但是起火有很多，对吧？我可以开暖气，可以添衣服，洗个热水澡，甚至我给你一个暖水，爱的抱抱，这都是不同的结果。甚至我累了可以休息，可以躺着，甚至可以再坚持一会儿，这也是不同的结果。但是你输入的时候，你只取一前一后一个因果关系就可以。而这两个因果关系就是在孩子的听觉当中一前一后的出现。同时你不要去解释因为所以，不要把因为，所以这两个词汇加进去那个因果小串联，那个因果串联那个 PPT 方式啊。\n我们做这个实际上就是给他说这个孩子的认知思维，这个体系的话，是一幢大楼的话，那么因果小串联就是砖头，或者说两个砖头的一种组合，那么我们呢？把因果串联简单的分了一下类，实际上就是给大家，嗯，开拓一下\"\n[7]      \"：举个例子啊，比如说粥，热粥烫到嘴的那种感觉，孩子会一下子把这种感受和“热”这个词连接起来。或者平时喝的饮料都是温的，突然给他一杯冰水，冷到他发抖，这时候你可以说“这水好冷”，孩子很快就能理解“冷”的含义了。\n家长：那气态的冷热呢？\n源爸：气态的话更多是环境上的，比如用吹风机的冷风、热风，或者是夏天家里开了空调，客厅很凉快，但某个房间没开空调，孩子一进去就会觉得“好热啊”。这些环境的冷热对比也可以作为样本输入。\n家长：生活习惯中怎么体现冷热输入？\n源爸：嗯，比如洗脚，平时用热水洗，突然换成冷水，孩子一脚放进去，就会觉得“今天这个水不对”。虽然冷水可能没冷到让他难受，但温度变化已经足够让他感受到冷热的差异了。你可以在这个时候说“今天水冷了”，然后再换一盆热水，做冷热的输入。\n家长：冷热样本还能结合哪些情境呢？\n源爸：还有一些带来明显不同结果的情境，比如冲黑芝麻糊，你只能用热水，冷水就冲不开。这种例子很多，像冲奶粉也是，用冷热能让孩子体会到差异。\n家长：还有其他建议吗？源爸：我们选择的冷热样本，最好是感受强烈的，比如冰水凉到让他发抖，热粥烫到嘴，或者是突然一股热浪袭来，这种情况下孩子对冷热的认知会更直观，输入效果也会更好。嗯，这些就是大概的思路了。\n家长：那么冷热的还是那一点，就是说如果你这个孩子是完全没有概念的，零基础，那么和许大人也是时候一样，就是你在输出的时候只抓住一个输出。不要这个太冷了，我们家里热水，现在热了，现在就不冷了。那孩子冷热他不知道你说到底没有状态啊。\n源爸：嗯，这就是我们要强调的\"\n[8]      \"等孩子掌握了具体颜色后，再逐渐引入“颜色”这种抽象概念。\n接下来我们讲第三个形容词——冷热。教冷热的目标是让孩子听懂“冷”和“热”这两个词代表的意思，而不是让他发展出解决问题的能力。比如，有的家长问：“我家孩子看到热水会自己兑冷水，是不是已经知道冷热了？”其实，这并不代表他听懂了“冷热”，我们的目标是让他理解这两个词的含义。\n冷热的样本可以分为三类：固体、液体和气体。固体：以食物为主，比如冷热的包子、面条等。\n液体：比如冷热的水、牛奶、饮料等。气体：比如冷热的风、蒸汽等。\n输入冷热的方法：空间上的对比：\n同时呈现冷热两种状态的物品，比如一个冷包子和一个热包子。让孩子自己摸一下，感受冷热的不同。比如，他摸到冷包子时，输入“冷冷的包子”；摸到热包子时，输入“热热的包子”。\n初期可以使用极端的样本，比如从冰箱拿出的冷包子或很烫的热水，给孩子留下深刻印象。\n时间上的对比：让孩子体验物品从冷到热或从热到冷的变化。比如，把冷包子放进微波炉加热，输入“包子热了”；或者把热包子放凉，输入“包子冷了”。\n意义样本的目的是让孩子体验到“冷的可以，热的不行”或“热的可以，冷的不行”的结果差异。比如：冷包子可以吃，但热包子太烫，需要等一会儿。热水可以喝，但冷水太冰，需要兑热水。\n我们可以通过日常生活中的场景和物品来输入和测试。比如：泡奶，用热水泡奶可以冲开，而冷水会导致结块，口感也不好。热水可以泡开面条，而冷水则会让面条结成一坨，不好吃。用热水浇花会把花烫死，这是一个后期可以使用的意义样本。洗完澡后用吹风机吹头发，可以输入\"\n[9]      \"我们的核心任务。我发现很多人把这当成了重点，其实90%的精力应该放在前面第一步——给他输入“大小”的概念，积累很多很多样本。所以，这个我就不多说了。\n我画了个图来解释为什么要铺垫。我觉得这个例子选得也不太好，大家看看就好，理解意思就行。比如，有个家长说：“我儿子对温度很敏感，稍微热一点他就不愿意。”那是不是有动机、有铺垫呢？其实他生活中确实体验过冷和热，也知道不喜欢太热或太冷，但这些只是体验，没有和语言链接起来。当你问他：“你要冷水还是热水？”他可能没反应。因为他听到的只是“甲”和“乙”，和之前的经历连不起来。\n所以，一开始不要直接问他要冷水还是热水。比如，你先给他一杯冷水，说：“哎，这杯水有点冰。”再给他一杯热水，说：“哎呀，这杯水太热了。”等他对冷和热有了概念之后，你再问他：“你要冷水还是热水？”这时候他听到的才是有效信息。\n如果铺垫不够，他可能对“冷”这个词有点熟悉，但具体是什么经历，他也记不太清楚。但如果铺垫足够多，他就会开始想：“哎，上次我随便选了热的，结果太烫了，吃了一个亏。”所以，铺垫很重要，不是单纯的体验，而是要和语言链接起来。当你跟他说一件事，或者给他选择的时候，这句话才是有意义的。\n第五步是抽象类的样本。我觉得抽象类的样本不用当成任务去做，生活中慢慢遇到就行。比如“大房间”“小房间”，你非要在学“大小”的阶段硬给他输入，其实很难。这些是需要随着认知积累慢慢发展出来的。所以，这些抽象类的样本，遇到就输入一下就好，不用太当重点。\n后面还有分类的方式，我再举个例子。比如沙发上东西太多了\"\n[10]     \"测试前先带他摸杯子感受温度差异，拿对了要立马给奖励。比如用冷水泡奶茶根本泡不开，热水才能泡，这种生活场景最直接。\n家长：冷热和其他温度词怎么区分？\n源爸：北方习惯叫凉水的，等孩子掌握基础后把凉和冷当同义词教。你看热水壶会冒气，冷饮瓶子挂水珠，这些特征多展示。就像进空调房突然哆嗦，这时候说“好冷穿外套”特直观。\n家长：温度变化怎么教更直观？\n源爸：空调房就是现成教材啊！大太阳下进商场冷得打哆嗦，立刻输入“冷”。家里留个没开空调的卧室，突然开门那股热浪扑脸，这时候说“这个房间好热”特真实。\n家长：日常怎么持续输入温度概念？\n源爸：水龙头左热右冷反复教，突然让他接冷水就是测试。吃东西前多问“要冷的包子还是热的？”，选错了让他摸微波炉加热的过程。对了，泡面用冷水根本泡不开，这就是现成教学道具嘛！\n家长：像“烫”“凉快”这些词会不会混淆？\n源爸：先确保冷热基础打牢。比如热水冒烟时说“小心烫”，冷饮瓶水珠说“凉快吧”。北方说“凉水”的，等冷热掌握后直接告诉“凉就是冷”，你看孩子洗澡突然碰冷水喊“凉！”，这时候接“对，这就是冷”完美衔接。\n家长：怎么帮助孩子理解温度相关的词汇呢？\n源爸：嗯，不用管，他说正常该怎么说的时候就怎么说，但你现在要围绕目标输入。比如他知道“烫”和“凉”之后，直接用“冷”“热”替换。比如电风扇没开他说“好冷”，其实想表达凉快，这时候直接说“凉快”就行，不用纠结。\n家长：孩子说错的时候需要立即纠正吗？\n源爸：不用纠正，对吧？比如他说“妈妈唱”其实是觉得烫，你就自然接一句“烫的”。他可能把“\"\n[11]     \"澡倒水太重了，需要帮忙，这是一种方式。那还可以，别的方式就我教你别的办法，打开筛子。\n方法这个词我们也特意去教过方法其实是一个很抽象的词了。然后他一开始教这个词是，好像是他的一个车什么坏了吧？我说我教你一个办法，你这样子弄车好了，然后他好像就把这个方法跟具体的这个事情，等于这个具体的好像是车轮装到车上，他就把这个东西方法就等于这个意思了，但后来经历越来越多样本之后，他就理解这个方法这个词。嗯，还有比如说试一下水温，试这个词。然后刚刚好，太冷了、太热了，然后是跟时间关联，其实时间其实想跟谁关联都能关联，就是效率的问题。所以适用于精熟之后用啊。\n昨天洗澡了，今天不洗澡。昨天坐的公交车，今天坐地铁吧。昨天坐公交车没座位，不坐了，今天我们打车去吧，就这种运用。然后还有独立，比如说独立洗澡，然后放水的时候，我教一下他那个左边是热水，右边是冷水，中间是温水，然后说，唉，要洗澡了，要洗澡了。然后先说完，我们说看完电视去洗澡吧，妈妈先去放水了，然后等过了一会儿放到了这里，然后等过了一会带他去洗澡，一看澡盆里空的，说忘记放水了。然后还有就比如说今天我运动了，出汗了，我身上很臭，我要洗澡。然后还有比如说玩泼水什么，就还有比如说当中还有那个浮和沉，就拿一些玩具，有的玩具是浮起来，有的是沉下去的，就都可以发散开来，想再往后，然后计划说这个计划是因为就是好多人问我怎么做计划的，其实是这样子的，就你们现在遇到的所有问题我都有遇到过。\n然后我们也不可能一下子说我用了一种方法，然后你们我最初试用的这个方法，然后你们也用这个方法就\"\n[12]     \"主动去厕所，这时家长可以先做示范，然后带他去外面尝试，让他逐渐适应。\n家长：第八个，有时候孩子坐在马桶上时，可以给他一些玩具。因为就像我们大人上厕所时也会玩手机一样，给孩子一些玩具可以让他们更愿意配合。\n源爸：第九个，当孩子专注在某件事情上时，尿裤子或拉在身上是很正常的事情。比如孩子在玩玩具或者看电视时，可能会突然尿裤子，因为他们很难自己意识到。遇到这种情况，不要责备他，而是带着他去换裤子，这是一个日常的处理方式。\n家长：我们有两个问题。第一个，我们家小孩现在已经两岁半了，之前一直用尿不湿。大概两岁左右的时候，他会自己大便后拍打，但过了一两个月又不会了。他还有一个问题，就是对身体接触比较敏感。比如我们给他换裤子或尿布时，他拉在身上也不让换，换尿布时也很难配合，每次都要花很长时间。\n源爸：那他现在能理解“尿尿”或“便便”这些词吗？比如玩偶提示他时，他能不能主动表达？\n家长：他能听懂，但不确定他是否完全理解。比如，他玩偶提示他时，他能点头，但如果我直接问他，他可能不会主动说。他能独立完成一些内容，但不是每次都这样。\n源爸：他是男孩对吧？他有没有尝试过站着尿尿？比如尿到小便池或者马桶里？\n家长：没有，我们从来没让他试过。去年夏天的时候试过让他穿裤子，他能自己尿出来，但之后就没再试过。我们还没做过这方面的引导。\n源爸：那回去可以先尝试引导他建立尿尿和便便的联系。比如，睡醒后带他去尿尿，告诉他睡醒后肯定会有尿意，带他去小便池或者马桶上尿尿。\n家长：我们都尝试过引导他，但他不一定有规律。他有时候半小时、一小时才\"\n[13]     \"，那么打开热水壶的开关或者烧好水后拔掉插头，也都是很自然的样本。这些具体操作取决于孩子的兴趣和能力。\n如果孩子的参与能力较弱，你可以从他感兴趣的地方入手。比如他喜欢玩水龙头，就让他参与接水；他喜欢玩开关，就让他操作开关。我列举这些样本并不是要求你在同一个事件中全部输入，只是给大家提供一些例子。\n再比如洗脚这件事。平时孩子通常用温水洗脚，但今天你准备了一盆稍冷的水。当他脚一放进去，觉得不舒服时，你可以说：“妈妈去拿热水壶。”然后拿着热水壶往盆里倒水，调整水温后再让他洗脚。如果你想再进行一次输入，可以倒掉一些冷水，让孩子拿着热水壶倒一次热水，这又是一次自然的输入机会。\n这和刚才提到的样本有什么区别呢？首先，这个事件可以继续发展。一方面，你可以通过这个事件输入其他名词，事件不变，只是更换目标词。另一方面，这其实为二阶段的形容词教学做了铺垫。比如，如果你想教孩子“冷”和“热”这两个形容词，这个事件仍然可以使用。孩子虽然可能还不理解“冷”和“热”，但他通过前期的体验，已经知道洗脚的水温应该是怎样的，现在这个水温让他不舒服。通过调整水温，他能直观感受到“热”的概念。有了前期参与事件的铺垫，再输入“冷”和“热”，孩子就能更快理解。\n所以，我们在形容词阶段的输入应该是“蜻蜓点水”式的，但这种“蜻蜓点水”是建立在前期丰富事件体验的基础上的。\n如果以热水壶为例，如果我们总是用类似“热水壶挡住了他的食物”或者“热水壶里装着他喜欢喝的东西”这样的方式，孩子虽然可能不理解，但这种方式缺乏可持续性。当你希望孩子继续发展时，你\"\n[14]     \"往盆里倒水，调整水温后再让他洗脚。如果你想再进行一次输入，可以倒掉一些冷水，让孩子拿着热水壶倒一次热水，这又是一次自然的输入机会。\n这和刚才提到的样本有什么区别呢？首先，这个事件可以继续发展。一方面，你可以通过这个事件输入其他名词，事件不变，只是更换目标词。另一方面，这其实为二阶段的形容词教学做了铺垫。比如，如果你想教孩子“冷”和“热”这两个形容词，这个事件仍然可以使用。孩子虽然可能还不理解“冷”和“热”，但他通过前期的体验，已经知道洗脚的水温应该是怎样的，现在这个水温让他不舒服。通过调整水温，他能直观感受到“热”的概念。有了前期参与事件的铺垫，再输入“冷”和“热”，孩子就能更快理解。\n所以，我们在形容词阶段的输入应该是“蜻蜓点水”式的，但这种“蜻蜓点水”是建立在前期丰富事件体验的基础上的。\n如果以热水壶为例，如果我们总是用类似“热水壶挡住了他的食物”或者“热水壶里装着他喜欢喝的东西”这样的方式，孩子虽然可能不理解，但这种方式缺乏可持续性。当你希望孩子继续发展时，你又需要围绕新的目标去绞尽脑汁地想样本。实际上，这种方式会让你明显感受到，这些事件和样本都是层层递进的。前面做的这些事情不仅在输入目标，也在为孩子后续的发展打基础，这一点非常重要。\n我以热水壶为例。通过事件教学，我会先想出相关的事件，再从中提炼具体的样本进行输入，用这种方式进行干预。而且，当你真正用这种方式干预了好几个月后，你会发现，只要你明确了当天要做的事件和目标词，在事件中很快就能想出要输入的样本。也就是说，这种干预方式已经内化为你\"\n[15]     \"冷”和“热”呢？\n源爸：你可以通过指令来测试。比如，让他拿一杯冷水或热水，如果他能够正确执行，说明他理解了。另外，水龙头也是一个很好的测试工具。通常，水龙头往左是热水，往右是冷水。如果他能够正确打开水龙头并接出热水或冷水，也说明他理解了“冷”和“热”。\n家长：那如果他通过视觉无法判断冷热，怎么办？\n源爸：你可以先辅助他摸一下，比如让他摸一下冷水杯，然后说“拿热水”。如果他能够按照指令拿对，说明他理解了。这种测试的前提是他之前多次使用过水龙头或接触过冷热水。\n在输入“冷”和“热”时，要注意孩子的反应。比如，如果孩子喝了一口很冰的水，你可以观察到他的反应，然后输入“这水太冷了，我给你换一些”。至于“热”的输入，取决于孩子的能力。如果你觉得他能理解，那就没问题。但要注意，不要在他感受到冷的同时输入“热”，这样会让他混淆。\n另外，输入“冷”和“热”时，要确保孩子确实感受到了冷或热。我之前有个案例，孩子对冷热不敏感，妈妈为了教他，把粥冻成冰碴子给他喝，结果孩子喝完后毫无反应。这种情况下，就不适合输入“冷”，而是要找其他更有感觉的样本，比如用冷水洗脚或洗澡，观察他的反应再输入。\n在做这个样本时，我建议你找个小凳子坐下，这样体验会更充分。视频中的孩子脚没有完全泡到水里，妈妈只是往他脚上泼了几下水。你可以找个矮一点的凳子，让孩子坐上去，脚完全放进水里，感受一下冷热。然后你可以说“水太冷了”，接着换一盆热水，用热水洗脚。\n源爸：大家关于冷热、颜色、大小，或者其他一阶内容还有什么问题吗？\n家长：老师，我有一个问题。\"\n[16]     \"水龙头或接触过冷热水。\n在输入“冷”和“热”时，要注意孩子的反应。比如，如果孩子喝了一口很冰的水，你可以观察到他的反应，然后输入“这水太冷了，我给你换一些”。至于“热”的输入，取决于孩子的能力。如果你觉得他能理解，那就没问题。但要注意，不要在他感受到冷的同时输入“热”，这样会让他混淆。\n另外，输入“冷”和“热”时，要确保孩子确实感受到了冷或热。我之前有个案例，孩子对冷热不敏感，妈妈为了教他，把粥冻成冰碴子给他喝，结果孩子喝完后毫无反应。这种情况下，就不适合输入“冷”，而是要找其他更有感觉的样本，比如用冷水洗脚或洗澡，观察他的反应再输入。\n在做这个样本时，我建议你找个小凳子坐下，这样体验会更充分。视频中的孩子脚没有完全泡到水里，妈妈只是往他脚上泼了几下水。你可以找个矮一点的凳子，让孩子坐上去，脚完全放进水里，感受一下冷热。然后你可以说“水太冷了”，接着换一盆热水，用热水洗脚。\n源爸：大家关于冷热、颜色、大小，或者其他一阶内容还有什么问题吗？\n家长：老师，我有一个问题。我们现在在一阶阶段，教他电风扇的多样本时，遇到了一些情况。比如，饭菜端上来很烫，他以前吃到烫的东西时，我会说“好烫”，然后他会自己收回手，用嘴吹一吹。现在到了洗澡时，我说“水好烫”，他也会对着水吹。这种情况是不是应该避免？\n源爸：你输入了也没关系。孩子形成了“用嘴吹一吹”是解决烫的方式的连结，这是正常的。你觉得他吹水奇怪，但这只是他的一种反应。\n家长：我们在教风扇多样本时，他知道大的风扇，我给他买了一个手持小风扇。吃饭时，饭菜很烫，我就\"\n[17]     \"的生活中，让孩子亲身体验，理解对错。\n其实，说穿了就是输入和测试，实际上就是一般小孩子最容易学的两个词汇，一个是“烫”，一个是“辣”，都是带来强烈刺激的感觉，基本上一两次他就明白了，而且有时候还想吃的东西好热的，太烫了。还有一个就是关于气体的词，实际上就是气温。那我们怎么操作呢？现在有空调比较方便，一个房间开空调，一个房间不开，制造出温差，或者利用汽车内的空调。然后当孩子从一个空间切换到另一个空间时，就能感受到温差。还有一些直观的例子，比如冷气和热气让他摸一下。这种输入一下就可以，但主要是在做这些输入时，大家一定要记住，不要干巴巴地叫孩子过来摸一下这是热，再摸一下这是冷，这样实际上没什么效率。要通过实际场景让他感受到“我很想要热”或“我很想要冷”的时候。\n另外还有一个细节，就是昨天有位妈妈提出来的，当我们在教这些对比的形容词时，你做输入的时候，一句话里不要同时包含多种感觉，就是当下什么感觉就是什么感觉，你不能说“哦，水太热了，妈妈给你倒凉水”，这样孩子会混淆，到底是冷还是热？\n这样就其实给孩子们输入比较清晰。好，关于冷热，后面我们看一下，那个大家都可以看一下这些，其实我们就是一些常见的一些样，记住就是当下孩子是什么感觉？输入什么感觉？那么现在还快到冬天了，将热水加起来。然后还有一个事情很奇怪，就是有很多我们还在是怕吹风机。那为什么他吹风机实际上就是说在小，因为吹风机你稍微一靠近，马上这个灼灼烧感很强，就很多小孩子就是说小时候他自己也不懂的，他也不知道怎么控制区别的。\n实际上都是一些操作的细节，就是像\"\n[18]     \"你可以用他已经理解的词来辅助他理解。比如，如果他已经理解了“烫”或“冰”，你可以用这些词来帮助他理解“冷”和“热”。举个例子，如果包子刚出炉，你可以说“这个包子是热的，很烫”，因为他已经理解了“烫”，这样你就在输入“热”的同时，帮助他建立“热”和“烫”的联系。\n家长：那可以说“热的包子很烫”吗？\n源爸：可以。对于“冷”和“冰”也是同样的道理。你可以在具体语境中使用这些词，比如“凉快”通常表示一种舒适的状态，而“冷”则可能让人感到不舒服。你可以在不同的情境中输入这些词，比如天气很热时，你可以说“这里好凉快”，或者在夏天的小溪边，你可以说“这水好凉快”。\n家长：那怎么让他理解“冷”和“热”呢？\n源爸：你可以通过制造反差来帮助他理解。比如，平时洗澡或洗脚时用热水，突然换成冷水，他会感觉到不同，这时你可以输入“水太冷了”。这种反差的体验可以帮助他更好地理解“冷”和“热”。不过，这种反差体验要适度，不能让他一直处于“水深火热”中，否则他会受不了。\n家长：那怎么测试他是否理解了“冷”和“热”呢？\n源爸：你可以通过指令来测试。比如，让他拿一杯冷水或热水，如果他能够正确执行，说明他理解了。另外，水龙头也是一个很好的测试工具。通常，水龙头往左是热水，往右是冷水。如果他能够正确打开水龙头并接出热水或冷水，也说明他理解了“冷”和“热”。\n家长：那如果他通过视觉无法判断冷热，怎么办？\n源爸：你可以先辅助他摸一下，比如让他摸一下冷水杯，然后说“拿热水”。如果他能够按照指令拿对，说明他理解了。这种测试的前提是他之前多次使用过\"\n[19]     \"当做一个选择项放进去就是了，你孩子能力怎么样？\n家长：我带他面试也过了。\n源爸：嗯，语言认知在几期，哪个阶段？\n家长：可能是二阶初。\n源爸：二阶初。说实话去幼儿园还是蛮吃力的，这个再说。\n家长：是 3 个小时托班那种。\n源爸：如果家里有条件的话，我不建议上托班，如果家里没条件，实在管不了，那你就放一放，因为托班其实就是一个混时间了。但是大人也要喘口气。所以看家庭情况而定，我也没一棒子打死那个后面。\n家长：这个选择事件我还是有点疑惑。就是输入的频率怎么掌握。\n源爸：选择事件的频率啊？什么时候用？我觉得你先做，你自己先给他梳理一下他每天做的事情，然后先给他做输入，然后你就可以用了，可能一开始少一点。一天来个一两次、两三次也行，就是最主要是符合生活。\n家长：就比如说洗澡的时候给他看一个洗澡的照片，还有其他做的事情，这样子给他选。\n源爸：那第一步你先要做输入，我们洗澡去，那一次我看一下照片，我们洗澡去了，做一段时间输入，然后再洗澡比如说和其他的东西做一个对比，然后问他，要洗澡还是要做什么什么。好，大家暂时没问题了，那么我们把选择疑问句讲完，选择疑问句就是选择场所选择、事件选择、人物选择、交通工具选择，最后一个是什么？多项就是三选一，四选一。那么选择疑问句我们基本上讲完了，我想告诉大家的是，就是当孩子理解选择之后，不要一天到晚的让他去选。很多人、很多家庭，到最后孩子一看你拿出两个东西，马上哎呦，烦的不行。真的是这样的，实际上我们比如说跟孩子交流的时候，我觉得选择只能占到30%，更多的直接跟孩子说如何做，我们\"\n[20]     \"就是被误解的动机。第一个就是认为动机就是强化物，就是最早几期家长就是容易出现这个问题，现在感觉大家已经都进步了，都比较少出现这个问题。然后另外一个就是目标词汇与动机不匹配。举个例子，比如说有个家长说他们家小孩就是刷牙的时候他很怕热水，就微微有点温，他就不喜欢，他就喜欢冷水，他就觉得很有动机。然后他等孩子刷牙的时候就问他，你要冷水还是热水？然后他发现小孩子是没有反应的，然后他就很困惑，他明明有动机，他明明很怕热，他为什么没有反应？大家想想为什么？你看就是我这杯水到我嘴巴里的时候，我发现它是热的，我不喜欢，但是我在刷牙这个场景里我可从来没听，第一，他不知道他不喜欢的这个东西叫热水。所以到这个场景你问他要冷水还是热水，他脑子里是没有东西的，他不知道你在说什么，他其实他有动机的，是他感觉到的那个东西，不是热水这个音，所以他是没反应的。这个就属于目标词和动机不匹配，其实怎么操作？其实肯定是先体验，先给他一杯热的，热的不要，我们要冷的，这些体验一定次数之后你再去问你要冷的还是热的，这个时候他才会有反应。另外一种不匹配，就是比如说有个家长说我们小孩特别喜欢拍球，很有动机，每天都要拍球，然后开始就拿球来教大小，然后问他，你要小拍小的球，还要大的球，小孩也无所谓，我随便拿个球我都能拍呀。那这个也是目标持不平，他有动机，是有拍球的动机，他没有大小的动机。所以我们的目标是大小我们就要想，比如我找一个大龙球，特别大和一个篮球，大球他拍不了，这个时候他才有大小的动机，我要去选一个小的，或者我拿一个乒乓球和一个篮球，乒乓球他拍\"\n[21]     \"，接下去的话我就是给他多输入那种，就是你昨天讲的复合指令。\n源爸：区辩型复合指令，这个是他需要及时强化的。\n家长：包括后面的那些形容词，其实我们在家也教了她很多，都是按照以前的教法，就是也有考过她，她有的时候反应也是蛮好的，就是我自己在教学过程当中我自己犯了很多错误，就是喜欢问问题。这个我昨天听你讲过之后，我意识到了这一点点，就是回家不能再一直就是重复性的问他的那种形式了。形容词像长短、大小、粗细、高矮胖瘦的话没有教过。其实大小的很多东西，我们就是在家是一颜色形状，这个也一直都在输入。\n源爸：那有没有理解的话，你要看东西的就是那种方长方形、正方形的，那种木块形的，它还可以，但是你要是弄圆形的大盒子，正方形的大盒子，长方形的大盒子，这个的话就有点不太。\n颜色的话，红、黄、蓝、红、绿。冷热的话，因为我们也一直跟他说不冷，热。开热水管。\n这个好像要没有那个什么推荐的，但是你给他冷水和热水这个才是可以的。他一摸他就知道了。主要是要冷，他能听懂。当然你也不用急，反正形容词我们下次我们会详细讲的，因为，然后区辩型复合指令，反正你也听到了，那么你现在就是说有些问题在什么地方？\n家长：然后我爸昨天就让我回去写一下，所以我就简单的写了一些，就是爸爸女儿最近在家的一些情况，给你们说一下吧。第一个就是我现在在家带他，因为他现在 4 岁半了，但是跟他一样大的孩子就是人家都在弄嘛？然后我就问人家你是怎么教的他们？\n源爸：我们得承认我们的孩子心智就是比正常孩子要弱，正常孩子比如说 4 岁半。那你的孩子现在我想想看他的心理年龄\"\n[22]     \"记忆能力提升后，他们不仅能记住任务步骤，还能在被干扰后迅速恢复注意力，继续完成任务。这种能力对孩子未来的发展至关重要。\n。\n事件教学中，有几点需要注意。首先是让孩子先体验和参与，再加入目标。这里举个例子：我之前有一个5岁多的孩子，他的妈妈在任何活动中都有很强的目的性，总是想给他输入一些目标。这个孩子以前上过机构，自我意识发展较慢，选择疑问句卡了很久，直到5岁多才突破。然后有一天他自己开酸奶，他自己把那个酸奶打开，想去尝试，想去喝或者怎么样。妈妈看到后，立刻抓住机会输入“外面”这个词。妈妈说“酸奶撒在外面了，来擦一擦”，怎么样的，说了一堆。结果孩子被说得心烦意乱，之后再也不愿意自己开酸奶了。这种过早、过于强烈的目的性输入，无形中给孩子带来了压力。\n我们建议，先让孩子体验和参与活动。比如在洗碗时，先让他玩水、吹泡泡，或者帮忙泡泡碗，这些都很简单。当孩子能够参与一部分活动后，再加入目标词。比如，目标词可以是“洗碗布”或者形容词“冷热水”。等孩子能够参与活动后，再把目标融入其中。过早加入目标不仅给孩子带来压力，还会剥夺活动本身带来的乐趣。\n对孩子来说，玩水、扯豆子、开水龙头这些看似是家务的事，其实都是游戏。但如果你过早加入目标，就会把孩子的注意力从活动本身转移到你的语言上，尤其是当我们说得太多时，孩子会觉得很烦。\n在事件教学中，不要急于输入目标，而是先让孩子体验和参与活动。当孩子熟悉并享受活动时，再自然地加入目标词。这样既能避免给孩子带来压力，又能让孩子在活动中保持兴趣和乐趣。\n第二点是关于逐渐撤销\"\n[23]     \"有些无动机的指令你也找不出什么自然结果的。你就放一放，你不要去非得逼着他做，反正他来求你的时候就可以了。就让他知道妈妈指令我是逃不掉的，总有在哪要等着我的。那我还不如现在去做掉。那么这里注意一个细节，你不要他拒绝你之后，你把巧克力拿出来了，这个叫贿赂。\n家长：在他有需求的时候\n源爸：对。反正他来找你肯定每天肯定很多次嘛。你想让妈妈干活，那你先听妈妈指令。\n家长：好。\n源爸：实际上刚才那个家长说的问题很常见的，这也是我们一个很常见的一个策略。首先我们自身要保证这个指令的合理性和便利程度，在这个情况下孩子还是不听，那就耗着。\n家长：你好，我想问一下，就是我们每次洗澡之前都会跟他说我们把浴盆抬到浴室里面，而且我确定他知道这个浴盆，然后他一直不配合。那我在想如果是耗着。\n源爸：他喜欢洗澡吗？\n家长：不喜欢\n源爸：不喜欢，换了我我也不配合的。这个不很正常的嘛，你明明不喜欢一件事情，别人非得让你去做，还要你配合，你当然不肯了嘛。\n家长：那晾晒衣服呢，他也是都不配合的\n源爸：简单，足够简单，我的意思是比如说晾晒衣服这件事情可能分成 10 个步骤，你让他参与其中一个。\n家长：对，我已经衣服已经拿在手上，他就拿下衣架。\n源爸：衣架放哪？\n家长：就在旁边，专门挂衣架的地方\n源爸：如果说你已经全部做到啦，他还不拿，OK，刚才那个方式处理。你就衣服放一放，你就别晾衣服了\n家长：关于听从，最近我家孩子老爱穿那个挺厚的那个鞋，这个春夏秋冬老爱穿那一双，我叫他穿新鞋。然后他不愿意穿，有时候跟他僵持一下，我说这鞋凉快，我确定他能\"\n[24]     \"比如我们今天，你们今天在这里上完课，妈妈在这上完课回去给爸爸讲，你们讲的肯定不是我讲的每一句话一模一样，讲回去你们肯定是消化完之后用自己语言去讲的。那他小孩子学也是这样子，他就是经历了很多很多之后，然后他开始才会内化为自己的语言，所以我们前期讲的那些转述，其实都是一模一样的转述。然后这个积累很多，很多时候小孩理想状态就是你给他讲一大段话，他其实会转化成自己语言讲，可能用两三句话讲出来。然后第三个是公交车三原则，这个就不细讲。然后另外一个就是被误解的动机。第一个就是认为动机就是强化物，就是最早几期家长就是容易出现这个问题，现在感觉大家已经都进步了，都比较少出现这个问题。然后另外一个就是目标词汇与动机不匹配。举个例子，比如说有个家长说他们家小孩就是刷牙的时候他很怕热水，就微微有点温，他就不喜欢，他就喜欢冷水，他就觉得很有动机。然后他等孩子刷牙的时候就问他，你要冷水还是热水？然后他发现小孩子是没有反应的，然后他就很困惑，他明明有动机，他明明很怕热，他为什么没有反应？大家想想为什么？你看就是我这杯水到我嘴巴里的时候，我发现它是热的，我不喜欢，但是我在刷牙这个场景里我可从来没听，第一，他不知道他不喜欢的这个东西叫热水。所以到这个场景你问他要冷水还是热水，他脑子里是没有东西的，他不知道你在说什么，他其实他有动机的，是他感觉到的那个东西，不是热水这个音，所以他是没反应的。这个就属于目标词和动机不匹配，其实怎么操作？其实肯定是先体验，先给他一杯热的，热的不要，我们要冷的，这些体验一定次数之后你再去问你要冷的还是热的\"\n[25]     \"说。他能独立完成一些内容，但不是每次都这样。\n源爸：他是男孩对吧？他有没有尝试过站着尿尿？比如尿到小便池或者马桶里？\n家长：没有，我们从来没让他试过。去年夏天的时候试过让他穿裤子，他能自己尿出来，但之后就没再试过。我们还没做过这方面的引导。\n源爸：那回去可以先尝试引导他建立尿尿和便便的联系。比如，睡醒后带他去尿尿，告诉他睡醒后肯定会有尿意，带他去小便池或者马桶上尿尿。\n家长：我们都尝试过引导他，但他不一定有规律。他有时候半小时、一小时才会有那种需求。可能是因为规律不好把握，因为他晚上睡觉时尿不湿会漏，但他晚上又不会主动去厕所。\n源爸：午睡起来后，因为天气不太好，我们一般在春夏季节解决大小便问题，那时穿的衣服少，也比较好打理。目前我们先从尿尿开始，因为尿尿比较规律，比较好掌握。基本上睡醒后他总会尿一泡，或者多喝点水后，半小时左右就会有尿意。\n家长：那大便呢？我们看看能不能找到规律，比如他两次大便大概间隔多少小时。如果能找到规律，就可以尝试每天固定时间带他去厕所。\n源爸：另外，也可以给他看一些关于上厕所的动画，让他对尿尿和大便有更直观的认识。一般来说，男孩尿尿时比较喜欢在空旷的地方，比如草地或田园，他们会更放松，更容易尿出来。\n家长：当然，不是每个家庭都有这样的条件，住在城市里不可能为了一次尿尿就去户外。所以一般还是在厕所解决。对于男孩，可以考虑用一些有趣的小便池，比如有风车转动或者有守门员扑球的那种。如果他没有不适感，他可能会更愿意去尿尿。\n家长：我记得有一个案例是用水冲的。他们家孩子在冬天的时候，\"\n[26]     \"，只要你的语言当中提到了颜色，不管是什么颜色，那么旁边一定要有其他颜色的同类物品的存在。就像我昨天说的这把灰色椅子和黑色椅子一样的道理，那么在传统机构当中，嗯，实际上它就出现这个问题，他会看到什么颜色，他就对孩子说这是什么颜色，那么这样的话，嗯，环境控制出了问题，导致这个孩子到最后会出现什么情况呢？他看到什么就说什么颜色，这是绿色的桌布，这是灰色的地板，这是白色的天花板、白色的灯。他就这么说的，你能说他说的错？他说的是对的，但是我们正常人怎么会这样说话！然后就会说自闭症，这样的就是自闭症，孩子不是这样的，纯粹就是教坏掉的，在桌面上教坏掉。所以我们在教形容词的时候，尤其是教颜色的时候一定要注意要有对比。\n那么我再问问大家，刚才我们不是教了冷热了嘛，对吧？如果没有对比，我光一杯热水放在那里，他一摸感觉到热了。我们能不能输入热，能吗？那没有一杯冷水做对比，我们为什么可以输入热呢？但是没有对比，其实是有对比的，跟整体跟他的记忆对比。就比如说我们看到一座很大的高楼，哇，这座楼好大，但是这座高楼边上没有小楼做对比呀。那我们为什么说大呢？因为在我概念里面，这个楼我第一次看到这么大的。实际上还是有对比，只不过它的对比对象是存在脑子里的，比如说你们，唉呦，这个好烫啊？那么实际上是跟他脑子里面那个常温的或者说稍微有点热的做对比。所以任何形容词都是有对比的，只不过对比一开始我们把它分成两类，空间对比和时间对比。但是颜色这个东西，它一般来说，尤其刚开始的时候，它就是空间对比。记住一个原则，当我们说这件事，我不管你在\"\n[27]     \"。与之前的修辞手法一样，这个过程也分为输入和测试两个阶段。在输入阶段，孩子会感受到热的感觉，并同时听到“热”这个声音。当然，可能之前孩子对“烫”比较敏感，能听懂“烫”，但对“热”可能听不懂，为什么呢？因为“烫”往往伴随着强烈的感受。如果孩子能听懂“烫”，但听不懂“热”，也可以通过第一次的交叉将“烫”过渡到“太热”。根据我们之前的讨论，我们需要提升输入效率。\n我们昨天一直在强调，要将孩子置于一个无理由的环境中。为了让孩子摆脱这种红利经济的束缚，他们必须了解关键词，比如“冷”或“热”。打个比方，当孩子满头大汗、气喘吁吁、口渴时，你给他一杯很烫的水。他说要喝水，但你给的是烫水，他当然喝不下去。这时，如果你是孩子，你会想到什么？你一定会想要一杯冷水，然后你慢慢领悟，恍然大悟，明白自己需要冷水。这次给你冷水，你会一下子接受。相反，比如他洗澡时，你打开水龙头或圆形喷头，喜欢用流动的水来复习，对吧？\n我记得以前北京有一位母亲，她曾问我一个问题。因为北京人早上习惯吃热气腾腾的包子，而晚餐时包子的温度则是从冰箱里取出的。她问孩子，是喜欢冷的包子还是热的包子。通过这种方式，母亲明白了孩子的偏好，即喜欢温暖的包子。这种通过选择疑问句，或是像我之前提到的喝水与不同水温的案例，都能实现高效的输入。\n如果进行测试，也可以通过指令来实现，比如像一般的饮水机，通常有热水龙头和冷水龙头。我们可以让孩子去开水龙头，选择热水或冷水，或者将一杯热水放在那里，让他去拿冷水。再比如，家中的水龙头往左转是热水，往右转是冷水，看看他是否能根据指令\"\n[28]     \"2025年1月7日 下午 12:17|2小时 49分钟 2秒\n关键词:\n孩子、疑问句、妈妈、饼干、形容词、爸爸、颜色、嘴巴、发音、认知、脑子、家长、玩具、自言自语、语言发展、指令测试、场景制造、小孩子语言\n文字记录:\n源爸：昨天我们讨论了两种大小的颜色。我们将这三个形容词放在最初的测试中，涉及冷热概念。实际上，我们是将内容分为三个部分：固态、业态和气态。固态部分通常与孩子有关的知识相关。基本上，我们都把这些内容写在上面的，并且我已读出来。与之前的修辞手法一样，这个过程也分为输入和测试两个阶段。在输入阶段，孩子会感受到热的感觉，并同时听到“热”这个声音。当然，可能之前孩子对“烫”比较敏感，能听懂“烫”，但对“热”可能听不懂，为什么呢？因为“烫”往往伴随着强烈的感受。如果孩子能听懂“烫”，但听不懂“热”，也可以通过第一次的交叉将“烫”过渡到“太热”。根据我们之前的讨论，我们需要提升输入效率。\n我们昨天一直在强调，要将孩子置于一个无理由的环境中。为了让孩子摆脱这种红利经济的束缚，他们必须了解关键词，比如“冷”或“热”。打个比方，当孩子满头大汗、气喘吁吁、口渴时，你给他一杯很烫的水。他说要喝水，但你给的是烫水，他当然喝不下去。这时，如果你是孩子，你会想到什么？你一定会想要一杯冷水，然后你慢慢领悟，恍然大悟，明白自己需要冷水。这次给你冷水，你会一下子接受。相反，比如他洗澡时，你打开水龙头或圆形喷头，喜欢用流动的水来复习，对吧？\n我记得以前北京有一位母亲，她曾问我一个问题。因为北京人早上习惯吃热气腾腾的包子，而\"\n[29]     \"一下是什么意思？先后我们的目的是让他听懂先怎么样后怎么样，或者先怎么样再怎么样，这句话是什么意思？学习先后的前置能力，就是实际上是两个动作，就是两个动词，一前一后两个动词，那么这两个动词，这两个指令他们要能听懂。这是前置能力，听懂的前提下，我们就可以输入先后。那么我把先后分成了两种，就是输入和测试输入阶段我们用逻辑性的先后顺序。什么叫逻辑性的先后顺序呢？就是这两件事情，你必须做了前一件事情，你才能做后面一件事情啊。这里有两个样本，大家能不能想出来逻辑性的先后有哪些样本啊？你说错了，这个不是逻辑性的先后，这是你认为的先后，不是孩子认为的先后，这个是对的。先脱衣服再洗澡。这个也不是，这个没有先后的，一定要先后的。\n先穿裤子在穿鞋。嗯，那不一定，看什么鞋不一定。那最简单的，你先把球捡起来再抛给我，先开盖子再喝水。先打开什么袋子再吃，先剥皮再吃，那么这种逻辑性的先后生活当中有很多很多，我们用逻辑性的先后做输入。\n然后逻辑性的先后输入一段时间，之后我们用并列性的先后做测试，并列性的先后就是谁先谁后都可以，这两件事情你先喝口水再吃饭，你先吃水果再吃饼干。\n什么时候你下一个并列性的那个先后的测试，孩子能做到了就算过掉了。\n家长：我有一个问题，我那个前几天，有这么一件事儿，就是他那个碗里的饭没了，我就说你先去找姐姐，再和姐姐说，我要盛饭。\n源爸：太复杂了吧？因为我们讲先后，我们是简单的两个动名词，就是他做的两个动作就可以了。\n家长 ：那就先去找姐姐再说\n源爸：她多大？\n家长：三岁半。\n源爸:我觉得正常孩子来说这个\"\n[30]     \"四种颜色。我可以教他“彩色”这个词吗？\n源爸：现在不用急着教“彩色”，先让他理解具体的颜色。等他掌握了红、黄、蓝、绿等基础颜色后，再逐渐引入“彩色”这种集合性概念。你可以通过对比黑白照片和彩色照片，或者黑白电视和彩色电视，帮助他理解“彩色”的含义。\n家长：有时候我说“红色”“蓝色”他知道，但说“蓝颜色”他就听不懂了。\n源爸：我们通常不会说“蓝颜色”，而是直接说“蓝色”。“颜色”这个词更多用于特殊疑问句或集合性概念，比如“这是什么颜色？”等孩子掌握了具体颜色后，再逐渐引入“颜色”这种抽象概念。\n接下来我们讲第三个形容词——冷热。教冷热的目标是让孩子听懂“冷”和“热”这两个词代表的意思，而不是让他发展出解决问题的能力。比如，有的家长问：“我家孩子看到热水会自己兑冷水，是不是已经知道冷热了？”其实，这并不代表他听懂了“冷热”，我们的目标是让他理解这两个词的含义。\n冷热的样本可以分为三类：固体、液体和气体。固体：以食物为主，比如冷热的包子、面条等。\n液体：比如冷热的水、牛奶、饮料等。气体：比如冷热的风、蒸汽等。\n输入冷热的方法：空间上的对比：\n同时呈现冷热两种状态的物品，比如一个冷包子和一个热包子。让孩子自己摸一下，感受冷热的不同。比如，他摸到冷包子时，输入“冷冷的包子”；摸到热包子时，输入“热热的包子”。\n初期可以使用极端的样本，比如从冰箱拿出的冷包子或很烫的热水，给孩子留下深刻印象。\n时间上的对比：让孩子体验物品从冷到热或从热到冷的变化。比如，把冷包子放进微波炉加热，输入“包子热了”；或者把热包子放凉，\"\n[31]     \"比如组合不同形容词？\n源爸：对，比如“拿大的红积木”或者“喝温的牛奶”。但前期先确保单个词掌握，后面再慢慢叠加。\n家长：现在教孩子冷热概念有什么具体方法？\n源爸：哈哈哈，是吧？比如吃包子时拿冷地让他用微波炉加热，过一两分钟摸摸看包子热了可以吃。洗澡时偶尔调冷水制造反差，比如突然用冷水浇一下说“好冷啊”，这种强烈对比效果特别好，但别长期用，否则孩子会崩溃的。\n家长：怎么测试孩子是否真理解冷热？\n源爸：摆冷热两杯水，说“拿热水”看他反应。测试前先带他摸杯子感受温度差异，拿对了要立马给奖励。比如用冷水泡奶茶根本泡不开，热水才能泡，这种生活场景最直接。\n家长：冷热和其他温度词怎么区分？\n源爸：北方习惯叫凉水的，等孩子掌握基础后把凉和冷当同义词教。你看热水壶会冒气，冷饮瓶子挂水珠，这些特征多展示。就像进空调房突然哆嗦，这时候说“好冷穿外套”特直观。\n家长：温度变化怎么教更直观？\n源爸：空调房就是现成教材啊！大太阳下进商场冷得打哆嗦，立刻输入“冷”。家里留个没开空调的卧室，突然开门那股热浪扑脸，这时候说“这个房间好热”特真实。\n家长：日常怎么持续输入温度概念？\n源爸：水龙头左热右冷反复教，突然让他接冷水就是测试。吃东西前多问“要冷的包子还是热的？”，选错了让他摸微波炉加热的过程。对了，泡面用冷水根本泡不开，这就是现成教学道具嘛！\n家长：像“烫”“凉快”这些词会不会混淆？\n源爸：先确保冷热基础打牢。比如热水冒烟时说“小心烫”，冷饮瓶水珠说“凉快吧”。北方说“凉水”的，等冷热掌握后直接告诉“凉就是冷”，你看孩子\"\n[32]     \"开门那股热浪扑脸，这时候说“这个房间好热”特真实。\n家长：日常怎么持续输入温度概念？\n源爸：水龙头左热右冷反复教，突然让他接冷水就是测试。吃东西前多问“要冷的包子还是热的？”，选错了让他摸微波炉加热的过程。对了，泡面用冷水根本泡不开，这就是现成教学道具嘛！\n家长：像“烫”“凉快”这些词会不会混淆？\n源爸：先确保冷热基础打牢。比如热水冒烟时说“小心烫”，冷饮瓶水珠说“凉快吧”。北方说“凉水”的，等冷热掌握后直接告诉“凉就是冷”，你看孩子洗澡突然碰冷水喊“凉！”，这时候接“对，这就是冷”完美衔接。\n家长：怎么帮助孩子理解温度相关的词汇呢？\n源爸：嗯，不用管，他说正常该怎么说的时候就怎么说，但你现在要围绕目标输入。比如他知道“烫”和“凉”之后，直接用“冷”“热”替换。比如电风扇没开他说“好冷”，其实想表达凉快，这时候直接说“凉快”就行，不用纠结。\n家长：孩子说错的时候需要立即纠正吗？\n源爸：不用纠正，对吧？比如他说“妈妈唱”其实是觉得烫，你就自然接一句“烫的”。他可能把“冷”和“凉快”搞混，但你不同场景输入对的词就行。比如吹风扇说“凉快”，摸冰水说“冷”，多说几次他就明白了。\n家长：这样输入近义词不会混淆吗？\n源爸：刚开始可能会，但你说人话就行。比如吃冷菜说“冷”，喝凉水说“凉”，他接触多了自己就区分开了。重点是你别刻意考试，就正常聊天。像前两天他说“电风扇好冷”，我回“对，电风扇吹着好凉快啊”，他就跟着学，哈哈哈。\n家长：需要系统教“冷热凉烫”的区别吗？\n源爸：不用！你越刻意他越懵。比如他觉得水烫，你\"\n[33]     \"的意思。所以，我觉得是需要大量的故事书去填补他这个空白。只有听故事或者怎么样生活当中碰到，而且包括这种高阶的认知绝对不是说一下能教得完的。你一定是某些时候碰到了触动他一下，碰到了，触动他一下。就光这个你我他吧，我儿子到现在为止大概刚刚能分清楚，但是还不能熟练的运用。光这个我你他，我大概用了大半年。但是我不是说直接把他叫过来，告诉他你我他怎么回事。我是日常生活中用一下，用一下，用一下。比如说，他要让我给他拍照。我就问他：“你拍还是我拍啊？”他说：“我拍”，我就把手机给他。他马上把手机给我说“你拍”。他要用门禁开去开车库的门，我问他：“你开还是我开”。他说：“你开。”我抢先一步我先开了，他跳死了 ，但是没办法。\n相对于正常教育来说，特殊教育其实它的整体教学水平真的很低很低。但是正常教育那么多年的发展，它已经是一套很成熟的体系了。我们孩子其实就是比别的孩子慢么，但是还是大量的借鉴正常孩子的教法。如果说这个孩子前置能力具备了，你教法正确了，一般一个星期就能看到效果。最多一个星期，他肯定能学会。如果说他还是学不会，要么前置能力不够，要么教法有误。这里插一句话，我认为桌面教学的教法是绝对错误的。先让孩子怎么样，然后再怎么样，然后才泛化。就算你顺顺当当教下来，你孩子和正常孩子的差距只能是越来越大。\nQ：孩子经常答非所问怎么办？而且，她好像都没有思考就乱说。\nA：你不要说你孩子了，我到孩子现在也经常这样，经常会跟我说动画片他所感兴趣的情节片段。谁让他能力弱呢？他想跟你沟通表达，他不说这个他还能说什么呢？别的他不会\"\n[34]     \"，我们经常用到区辨复合指令。\n我们后面形容词汇讲的比较快，然后我们会放一些录像给大家看，多少大家还有疑问嘛？没疑问，我们就直接往后面讲了。\n干湿的话其实无非也是一个对比，但是如果我们按照金字塔来推的话，先给他一些事件，关于干湿的事件，比如说他的衣服湿了，他这个经历有没有过？给他换上干的了，这个经历有没有过？而且我们再想一想，什么东西只有干了才能用，什么东西只有湿了才能用？有没有让他穿过湿的衣服让他有种不适感？这些让他先经历过，然后就是输入和测试。输入的时候手心出汗，就像是沾了水一样。或者拖把弄湿了，还有尿湿了裤子，我们得换条干的裤子，然后顺便洗洗手。把这个小手弄湿，我们才能打肥皂。如果你想让他体验这种感觉，你也可以试试，手干的时候就给他擦肥皂，擦几下发现不行，然后用水把手弄湿，这时候肥皂就能抹上去了。\n这种其实都是很好的样本，但是我们家长最容易犯的错误就是我们家长的习惯太强大了，我们从来可能都不会想到拿干的手去擦肥皂这件事情。但实际上体验过孩子就有印象。那么最简单，我们制造出一些偶然事件，你就端了一杯水，然后啪嗒一下掉在地上了，地上湿了。这就是一个很好的输入样本。\n有时候妈妈头发湿了，把妈妈吹吹干，这个衣服湿了，我们拿吹风机吹吹干。就是，我希望就是你们当孩子接触到干湿的时候，会有后续的一个解决方案。这里说起吹风机，我就我多说几句，吹风机这个事情很多自闭症孩子都怕，甚至说怕吹风机的声音或者是怎么样，大家有没有想过为什么怕？很多自闭症孩子都很怕的，为什么？大家有没有想过这个问题？实际上我告诉你，怕的\"\n[35]     \"关键原则：\n每次只聚焦1个区别要素（位置/干净程度/图案）。\n纠错时不否定孩子（不说“错了”），而是正向引导：“我们一起找找盘子里的...”。\n保持环境控制稳定性（连续3天使用同组对比教具）。\n家长：明白了，谢谢老师！\n家长：源爸，我家孩子特别喜欢跟我一起洗衣服。前几天我输入过“洗衣液”这个词，今天洗衣服时，他还没等我开口，就自己把洗衣液拿给我了。这是不是说明他已经掌握“洗衣液”这个词了？\n源爸：不一定。我们的目标是让孩子听懂“洗衣液”这个词，而不是仅仅知道要拿这个东西。建议测试一下：\n在洗衣液旁边放一个干扰物（比如肥皂）。\n说“去拿洗衣液过来”，观察他是否能准确选择。\n如果他拿对了，再输入“对，这是洗衣液”。\n家长：还有个问题，关于区别性指令。我家孩子想喝水时，总是只说“水杯”，不说具体是哪个杯子。我特意制造了一个障碍：拿了一个没水的杯子给他，其实我知道他想要的是每天带出门的保温杯（带盖子、带吸管的那种）。\n源爸：这里有两个问题需要分开解决：\n喝水需求：如果他真的渴了，给他其他杯子（装水）他会喝吗？\n杯子偏好：如果他只认准特定杯子，说明形成了固定思维模式。\n家长：他如果渴了，给他其他杯子（装水）他不会喝，但如果装可乐他会喝。\n源爸：这说明他并不是真的渴，而是对杯子有特定偏好。建议逐步调整：\n在家解决偏好问题：\n把原来的保温杯收起来，不让他看到。\n提供其他类似的保温杯（带吸管），慢慢过渡到不同类型的杯子。\n外出时：\n先在家解决偏好问题，再尝试在外使用其他杯子。\n关键点：\n不要让孩子形成固定思维模式\"\n[36]     \"的大小了。那么有时候比如说他说话声音小一点，声音大一点它能够提高音量，或者说声音小一点吵死了它能够压低音量，就说明这都能听懂。但是开始的时候我们就是用这种东西就可以了，让它调一下，然后是那个水流，水流的大小其实就是洗澡的水龙头，洗澡的莲蓬头和那个洗手的水龙头，反正现在家里面就是说我们在教孩这个概念的时候，我一再的强调这个原则，我们最好是把孩子放到一个不利的境地，然后他为了摆脱这种不利的境地，然后使用到那个概念。比如说刚才我说形状的大小，我们吃巧克力了，然后你给它切那么一点点，你给他，然后你自己吃一大块的，然后你看他什么反应？如果孩子不愿意，哼哼啊哼哼，这个时候你恍然到，噢，明白了，你要吃大块的巧克力，然后你给他那块大的，他的印象是非常深刻的，然后大那个概念一下子就有了。\n同样水流的大小洗澡了，然后你莲蓬头就开那么一点点水一滴一滴的流下来，他不舒服？怎么了？我们洗澡呀？你就装傻吗？噢，明白了，明白了。你要水开大，然后一边调大水。哗，这个水龙头开大，然后这种感觉就大的感觉就一下子就给到他了。实际上这就我刚才说的这些，其实就是符合了一些就认知原理的一些样本，那么这种样本比你那个在桌面上认大小卡片一千次、一万次都管用。他就这么一次他就能记住了。因为我们之前也就是说教了那么多孩子，后来就发现就是说有同样一个孩子，有的概念他学起来很快，有的概念他学起来很慢。然后我们就是去研究嘛？我们就发现存在的那么一些意义样本，或者说就是我刚才说的那种，那么基本上还这一两次他就记住，包括我们后面学的形容词，你知道孩子学哪个\"\n[37]     \"内容？\n源爸：上午主攻颜色教学实操，下午安排形容词进阶加案例设计。对了周日下午有专场答疑，你家娃要是卡在三阶段形容词上，到时候带具体问题来！\n家长：老师你好，我们家也才开始教颜色，然后就像视频上那样，可能今天教什么颜色就会穿什么样的衣服，嗯，我不知道这样做对不对哈？比如让他拿绿色毛巾时说“和你衣服一样颜色”，这样对吗？\n源爸：嗯，可以，这个就是给他提供参照物。你说“拿绿色毛巾，和你衣服颜色一样”，用衣服颜色做提示是合适的，这就是色卡配对的实际应用，这样操作没问题。\n家长：嗯，老师你好，袁妈说形容词是长期过程，比如大小需要30个样本，颜色和大小能同一天教吗？\n源爸：嗯，对，完全可以同步进行。比如今天教“大苹果”和“红苹果”交叉输入，但要注意形容词样本别超过三个，不然容易混淆，对吧？\n家长：那频率怎么把握？像之前教动词那样密集吗？\n源爸：嗯，一开始每天3—5次高频输入，比如洗澡时反复说“热水”“冷水”。但等他会说“烫”之后，你就不用刻意教了，日常遇到时自然提一嘴就行，嗯。家长：教冷热具体怎么操作？您刚举的包子例子没太明白，嗯。\n源爸：比如他平时吃热包子，有天你故意给冷藏过的。他一咬皱眉就说“冰包子！冷！”，这种反差体验比光看卡片深刻多了。就像孩子摔疼一次就永远记得“疼”这个词，对吧？\n家长：样本设计要和生活关联吗？比如用他怕的辣来教？\n源爸：嗯，对！要是孩子被辣哭过，你指着眼药水说“这个辣，不能碰”，他马上懂。形容词教学关键是要有强烈感官刺激，最好能调动情绪记忆，嗯。\n家长：那后期需要增加复杂度吗？\"\n[38]     \"新名词输入不是主要目标。孩子能力提升后，很多名词可以通过简单的方式输入。比如，你提到的“锅”的例子，可能只需要一两个样本，孩子就能理解。干预的目的是让孩子听懂更多、参与更多、学会更多。\n源爸：洗澡的例子，你说名词都输入完了，我不相信。你可以看看微信群里的思维导图，同样的一个事件，随着孩子能力的不同，我们说的台词可以不一样。一开始可能只输入一两个名词，逐渐延伸到形容词和因果串联。每次目标不同，表达的重点也不同。\n源爸：我们带孩子体验新活动时，首先要让他参与进来。成功参与后，再逐步输入目标词。最后，要把事件放手给孩子自己做。比如洗澡，等他能力提升后，可以一边洗澡一边聊天，最终让他自己完成。\n家长：我可能太程式化了，每天重复同样的内容，比如洗澡时说“眼睛、鼻子”，后来发现他都会了，我就不再说了，让他自己擦。这样既让他学会了洗脸，也听懂了我的话。\n源爸：大家看导图时要注意，我们单拎出事件是为了避免两个极端：一是除了目标词，生活中什么都不说；二是把所有目标词集中在一个时间段说完，然后让孩子放羊。我们要把目标分散到一天的不同时间和活动中。\n源爸：有些家长看到导图后，觉得自己要输入的东西太多，结果变成流程化的东西，孩子对妈妈的语言反馈越来越差。其实，我们要惜字如金，关键时候说一句话，孩子通过观察和体验学习，逻辑性会更好。不要矫枉过正，导图是为了梳理思路，但不要说得太多，把体验的机会留给孩子。\n源爸：家长需要清楚孩子已经掌握了哪些词汇，哪些词汇还不熟悉。针对不熟悉的词汇，设计事件时要有重点，不要盲目输入。比如，孩子\"\n[39]     \"还是热？\n这样就其实给孩子们输入比较清晰。好，关于冷热，后面我们看一下，那个大家都可以看一下这些，其实我们就是一些常见的一些样，记住就是当下孩子是什么感觉？输入什么感觉？那么现在还快到冬天了，将热水加起来。然后还有一个事情很奇怪，就是有很多我们还在是怕吹风机。那为什么他吹风机实际上就是说在小，因为吹风机你稍微一靠近，马上这个灼灼烧感很强，就很多小孩子就是说小时候他自己也不懂的，他也不知道怎么控制区别的。\n实际上都是一些操作的细节，就是像这种冷热的话，就是我们在生活当中给孩子输入的时候不要说。嗯，这个水太冷了，给你换一杯热水，就是在初期的时候是不要加入这样的因果。\n就是说你给一开始就冷，热的时候，你要契合当下孩子的感受，比如说洗澡的时候，孩子不愿意去，怎么啦？太冷了，然后你再把现在热给他。\n就是中间要有停顿。你不要冷热放在一句话。\n家长：然后，我家孩子对于冷热的感觉应该是相当敏感的。比如，他洗完澡出来，身上还挂着水珠时，就会说：“哇，好冷啊！”对于这种情境，他在表达时显然是能感知到冷热的。在这种情况下，我在输入相关信息时，可以适当地加入一些因果关系的描述。首先，需要增加更多关于冷热的样本，这样他就能更清楚地理解温度的变化。毕竟，他对冷热的感知也是通过实际体验，比如开空调来调节室温，从而逐渐掌握的。\n源爸：那么加入因果的话，我们要加入人为，或者说不仅仅是这里，其实还有其他的。\n家长：源爸问一下，就是卫生间那个水龙头，我可以两边那个字比较显眼的。\n源爸：嗯，不过我们想一想，就是除了这个字之外，实际上还是\"\n[40]     \"呢？不仅仅是光是形状的大小，实际上我们只接下来还有比如说声音的大小、水流的大小、风量的大小。那么声音的大小我们用那种，比如说那种手机上面的那个音量调节。还有再比如说像放音乐的，或者说蓝牙音箱调节，那么当你说大的时候你把这个推上去，小的时候关下来，这就是输入，我们放大一些，放大一边说大一边哗，音量起来，一边缩小一边音量按下去呃。测试的话，让孩子能做到这一步就行了。\n你把声音关小，把声音开大，它能够做到就可以了。包括电视遥控器上面的音量的大小。但是电视遥控器有个问题，上面按钮太多了，孩子是搞不清楚的，那么你可以就是音量大小上面用水彩笔涂一下，那个按钮。水流大小主要是水龙头开大，水龙头关起啊。有时候你在接水，你故意浮一点点水，你把水龙头开大，看看它能不能执行，就说明能听懂了。那么再比如说洗澡的时候淋浴嘛？你就给他用一点点水，你看看他有什么反应，他可能哼啊哼啊哼，怎么了？怎么了？妈妈明白了，你要把它开大啊。现在开大了。那么这种输入实际上是很高效的。那么分量本反正这边样本其实道理一样的。\n有时候小孩子喜欢玩那个风车，那你小小的风可能带不动，你开大风，这个风车就转的快。大小实际上就是它的概念是比较广泛的，那么我们一开始就形状的大小最重要，然后像声音、水流、风量这个是其次，也是生活当中经常会用到的。再往后什么数字的大小，那个东西咱们就先不要管咱，咱们不要去管管这些。后面我们学了数量的时候，再作为数量的大小的样本再提出来再引用。嗯，无非就是让孩子能够抓取这个，大就是这种开阔的感觉，小就是这种压缩的感觉。那么同样的\"\n[41]     \"。长期这样，孩子会越来越难以理解。\n我写了一个新的示例，关于热水壶的样本。有家长问我，孩子好像不喝奶粉，想不出太多样本。其实问题在于思维没有转变。你还在想着如何把样本和孩子的动机挂钩，比如热水壶里有他喜欢喝的东西，或者热水壶挡住他喜欢的食物。这样的设计操作起来很别扭。\n你需要转变思维：围绕目标词，先想出事件，再设计样本。比如热水壶，它主要用于烧水、泡奶、泡果汁或者喝水。孩子从外面回来很渴，或者在家运动后想喝水，热水壶里有水，或者需要倒水。这些都是热水壶的自然样本。借助事件，你可以想出符合生活情境的样本，而不用总是绞尽脑汁去和强化物挂钩。\n这种样本并不奇怪，但对于能力较弱的孩子来说，可能需要作为切入点。你可以问问旁边的家长，这种样本在实际操作中效果如何。我用过这种方式，觉得还不错，但它只是一个点。我们希望把干预发展成一个可持续的过程。\n\n在烧水这个事件中，关于热水壶的样本可以包括：打开热水壶盖子、往热水壶里接水，或者操作热水壶的开关。有些孩子喜欢拨弄开关、插头之类的东西，那么打开热水壶的开关或者烧好水后拔掉插头，也都是很自然的样本。这些具体操作取决于孩子的兴趣和能力。\n如果孩子的参与能力较弱，你可以从他感兴趣的地方入手。比如他喜欢玩水龙头，就让他参与接水；他喜欢玩开关，就让他操作开关。我列举这些样本并不是要求你在同一个事件中全部输入，只是给大家提供一些例子。\n再比如洗脚这件事。平时孩子通常用温水洗脚，但今天你准备了一盆稍冷的水。当他脚一放进去，觉得不舒服时，你可以说：“妈妈去拿热水壶。”然后拿着热水壶\"\n[42]     \"我们判断他是不是理解的依据。尤其是越到后期越难测试，我们就越多的要根据他的反应和的语言表达来判断他是不是理解了某个词汇。\n其实啊，刚才那个视频前面还有一个，就是两个小朋友在玩车，或者爸爸在那儿玩车，一边玩一边念叨着“车子慢慢走，车子快快走”。但对小朋友来说，这些话好像没啥特别的感觉。他们就是拿着车，听爸爸妈妈说这些话，对他们来说，车子走得快还是慢都差不多，他们就喜欢推着车玩。所以啊，当他们玩车的时候，他们真正在意的不是车跑得快不快，而是他们自己在玩车的那份乐趣。\n这个视频挺有意思的，它展示了什么是“快”。想象一下，外面下起了大雨，你得赶紧带着孩子跑回家。如果孩子对这个场景印象很深，那他可能一下子就明白了“快”是什么意思。就像我们用跑步机来教孩子理解速度一样，孩子玩得多了，就能通过他的反应看出他是否懂得快慢。比如，你把跑步机调快，看他能不能马上按下正确的按钮，让跑步机加速。这和我们教孩子分辨冷热是一个道理，不一定非要让孩子直接接触冷水或热水。有时候，你可以在洗漱时先调好水温，然后告诉孩子，现在我们要加点冷水或热水，看他能不能把水龙头转向正确的方向。这些都是判断孩子是否理解的好方法。所以生活中这种判断，他是不是能够听懂的机会是很多的。\n拿重的水壶浇花，拿重的水壶，这个是轻的，水壶里面没有水，拿这个重的水壶，这个里面有水。\n在一开始看到这个视频的时候我也有点懵，因为我们正常人判断我也会以为大的这个水壶会比较重，因为我们的生活经验告诉我们，一般通常来说这种大的会比较重。那妈妈可能想做一个比较反常规的，是让他\"\n[43]     \"可以的，完全可以。你可以选择日计划，也可以做周计划。关键是找到最适合自己的方式。很多家长会觉得做周计划很难完全落实到位，而日计划可以更具体地操作，效果也会更明显。\n家长：那对于干预初期，怎么保证效果呢？\n源爸：干预初期，我们要求大家多做样本设计，去寻找输入的机会。一开始家长可能很难找到这种节奏，感觉不到最初的设计样本的思路。我们会先帮助大家提升这方面的能力，之后再逐步增加任务和目标。对于新词的输入，连续性是很重要的，但刚开始时直接跳跃到这一步，家长确实会觉得有挑战。\n家长：那目标设计上有没有什么特别的要求？\n源爸：目标设计上要围绕着孩子的实际情况，不必一次性要求过高。逐步增加输入的次数和任务，确保在实际操作中孩子能理解并执行。\n家长：嗯，去玩或者洗澡，但是你这个思路还是会有一些问题的。你把我发在那个群里面的表格打开一下。你一开始这样做的话，会有一个问题，就是会导致样本的数量和网络不够。输入衣服袜子每天起床一次，衣服袜子就结束了，那多样本就拆了。所以说我们这样铺垫不是说哪个是对的，哪个是错的，而是说让大家循序渐进地学会这种方法。我发到13那群里，就是我罗列了一些事件，然后涉及你的一些名词的梯度嘛。形容词我没写，因为你的交到形容词阶段一个礼拜，两个礼拜可能也就那么两个形容词，所以说你就。到时候把形容词往里面一放就可以了。\n源爸：嗯，先举一个例子，举一个起床的例子，比如说起床的最直接相关的可能就是衣服和裤子。但是如果我们一开始就这样教的话，很可能你这一天就是关于衣服和裤子，就起床和睡觉，就这么两次输入。对\"\n[44]     \"饼干而不是黄色饼干，可以简单解释为“我不想吃黄色的”。不用太纠结这个问题，重点是孩子能根据颜色指令正确执行。\n家长：那如果孩子对黄色杯子有偏好，导致测试无法进行，怎么办？\n源爸：如果测试失败，可以灵活调整为输入。比如，孩子拿了黄色杯子，你可以说“红色杯子不要了，我们用黄色杯子喝”，同时输入“红色杯子”。如果孩子对某种颜色有强烈偏好，测试时可以换其他样本，比如红色和黄色的盒子或包，避免使用他有偏好的物品。\n测试是通过指令判断孩子是否能听懂颜色。比如“拿红色的杯子”。输入是通过自然情境帮助孩子理解颜色。比如“我们用红色杯子喝水”。视频示例：\n在视频中，妈妈让孩子“把套娃放在红色套娃里”，这是一种输入，因为只有红色套娃一个选项。如果是测试，旁边应该有其他颜色的套娃作为干扰。更好的输入方式是配对，比如拿着红色套娃的上半部分，让孩子拿红色套娃的下半部分，然后组装起来。\n教颜色时，可以利用带有颜色的容器，并在其中一个容器里放入孩子的强化物（比如饮料）。比如，准备红色和蓝色的杯子，红色杯子里装饮料。当孩子看到两个杯子时，输入“红色杯子里有饮料”或“拿红色杯子喝饮料”，然后拿起红色杯子喝。这样，孩子会将颜色与动机关联起来，更容易记住颜色。\n如果孩子对测试物品没兴趣，可以换其他强化物。重点是让孩子通过动机理解颜色。比如，用他喜欢的饮料或零食作为强化物。\n色卡辅助的目的是在孩子不理解颜色时提供提示。比如，让孩子“拿蓝色毛巾”，如果他听不懂或拿错了，可以拿出蓝色色卡，辅助他拿对。但不要在指令刚发出时就拿出色卡，否则孩子会\"\n[45]     \"他们自己在玩车的那份乐趣。\n这个视频挺有意思的，它展示了什么是“快”。想象一下，外面下起了大雨，你得赶紧带着孩子跑回家。如果孩子对这个场景印象很深，那他可能一下子就明白了“快”是什么意思。就像我们用跑步机来教孩子理解速度一样，孩子玩得多了，就能通过他的反应看出他是否懂得快慢。比如，你把跑步机调快，看他能不能马上按下正确的按钮，让跑步机加速。这和我们教孩子分辨冷热是一个道理，不一定非要让孩子直接接触冷水或热水。有时候，你可以在洗漱时先调好水温，然后告诉孩子，现在我们要加点冷水或热水，看他能不能把水龙头转向正确的方向。这些都是判断孩子是否理解的好方法。所以生活中这种判断，他是不是能够听懂的机会是很多的。\n拿重的水壶浇花，拿重的水壶，这个是轻的，水壶里面没有水，拿这个重的水壶，这个里面有水。\n在一开始看到这个视频的时候我也有点懵，因为我们正常人判断我也会以为大的这个水壶会比较重，因为我们的生活经验告诉我们，一般通常来说这种大的会比较重。那妈妈可能想做一个比较反常规的，是让他根据这种身体感觉去判断出来的。但是我们在一开始教轻重的时候，像这种其实虽然你测试了，但是输入的初级阶段，我们应该控制好变量，尤其是教某一类形容词，尤其是孩子卡住的形容词，一定要控制好干扰因素，不管输入还是测试都好，我使用的样本，比如说轻重两个水壶，那么我会选择两个一模一样的水壶，就是这两个水壶一模一样到什么程度呢？就是只有轻重不一样，其他所有的都一模一样，这样子孩子他才能够更精准的去抓取到这个轻重的特征。\n这个轻或者重到底指的是这个水壶\"\n[46]     \"，梳理一下，还有哪些样本可以补充。就像上午说的，即使孩子已经会了，还是补充越多越好。\n第二步是寻找生活应用场景。比如前面提到的“口子大小”是一个样本，但结合不同场景，又可以变成很多样本。比如吃饼干时包装的口子大小，倒酱油、倒醋时瓶口的大小，吃饺子时蘸醋的口子大小，还有孩子咳嗽时喝的糖浆，瓶口太小倒不出来，用剪刀把盖子剪开，口子大了就一下子倒出来了。\n结合不同的场景，你会发现样本数量会迅速增加。比如前面有10个样本，再结合不同场景又能变成100个。关键是要学会用分类的角度去思考。比如“水流大小”这个样本，洗手、洗脸、洗脚时水流大小都不一样，这些都是可以输入的场景。接水、浇花、刷牙时也可以输入。比如故意把水流开得很细，让孩子接半天，然后告诉他水太小了，可以开大一点。刷牙是一次输入，浇花又是一次输入。不过这些输入不要集中进行，生活中的点滴都可以慢慢补充，这样既能让孩子不断复习，也能让样本更丰富，理解更准确\n说到分类，我可以再说一个例子，就是分类这种思考方式是真的是很好用的，比如说词汇结合，我们也开始凭空想，也很困难的。你说我今天浇了水，我喝水，我接点水去浇花，好像也就这些了，想一想噢。嗯，那我们可以就可以想用分类的方法去想呀。有水，我从人身体部位开始想洗脸要用到水，刷牙要用到水，洗手要用到，那洗手都在家里用到水。我们比如说出去玩，我今天要出去玩。我也想想，哎，出去玩你要吃东西，你手脏，你带瓶矿泉水呗。哎呦，妈妈这有水呢，奶奶冲冲手是吧？也是一次输入，也是一次很好的输入。不是洗手，他脑子里的概念不是，只是说\"\n[47]     \"店吃饭了，给他看一下照片，到了那里我们到早餐店就完了。\n家长：这事情他之前就是带去的时候，他可能就是影响的因素很多。\n源爸：不要去管影响因素，不要管他看不看，我说了他爱看不看，但是我们只管输入就行了。\n家长：好，嗯。\n源爸：所以这也就是我昨天一直强调那种输入，我们不要求孩子有什么反应，输入是一个单向的。\n家长：源爸，你好，就是我们小明会有一个问题，就是他对事件的理解可能不是那么深刻，要不要去测试他听得懂这个事件？\n源爸：听得懂这个事件，就说一下，看他会不会吧嗒吧嗒的去，有反应吗？就比如说洗澡，走我们去洗澡啦，看他回应，于是。然后你跟他观察一下他的反应，他如果喜欢洗澡的，他反应开心，如果不喜欢洗澡的就哭丧了嘞，没反应。没反应就做链接了，你就输入了，因为你孩子还小呀，小的话我们是我们主要输入一下就可以了，而且是蜻蜓点水式的输入就可以。\n家长：嗯，好好，谢谢。2025年1月7日 下午 12:25|3小时 21分钟 48秒\n关键词:\n孩子、裤子、妈妈、衣服、词汇、老师、认知、小朋友、爸爸、控制环境、ipad、语言辅助、环境设置、样本设计、目标名词、发展水平、名词链接、语言逻辑\n文字记录:\n那下午就给大家讲一下，就是我们接下来，这次培训之后回去大家要做哪些事情？这里是默认孩子都是零基础的情况下去做的一个这样的表格，这个就是我们第一个月我们要做的一个目标的形式，我们回去第一件事情要去找目标，找目标\"\n[48]     \"的排列兴趣，又能通过每日高频接触自然习得核心词汇。\n家长：老师，我发现孩子两个月只掌握了20-30个词汇，进展是否过慢？另外孩子存在挑食问题，我们试过饥饿疗法但效果不佳。\n源爸：进展缓慢的核心在于家庭干预方向偏差。关于进食问题，真正的解决方案不是饿两天，而是建立正确的饮食规则——在固定餐椅、固定时间段内提供食物，15分钟后无论吃多少都收走。重点是要全家人统一执行，特别是老人不能偷偷喂食。我们曾介入过一个案例：奶奶总在饭后塞饼干，导致孩子正餐永远吃不好，直到全家达成协议才改善。\n家长：确实存在老人过度保护的情况，比如不让孩子接触玻璃器皿只给塑料玩具，这种教养方式需要调整吗？\n源爸：必须调整。过度保护会严重限制能力发展，建议逐步进行安全教学：先用塑料杯装温水，掌握后过渡到玻璃杯装冷水，最后用玻璃杯装温水。就像学走路必然会摔跤，我们要控制风险而非杜绝尝试。\n家长：关于指令执行的问题，孩子有时不执行明确理解的指令，该如何处理？\n源爸：这需要系统性分析，举例说明：如果\"把积木给爸爸\"的指令成功率50%，首先要检查：1.是否每次完成都有强化（如击掌+口头夸奖）；2.指令场景是否单一（只在桌边进行）；3.是否存在干扰因素（旁边有他喜欢的玩具）。建议从简单指令开始重建听从，比如将\"把红色积木放进盒子\"简化为\"放积木\"，成功3次后再增加难度。\n家长：实际操作中遇到生理限制怎么办？比如教2岁孩子用锅接水，他够不到台面。\n源爸（：及时调整教具尺寸。可以用儿童厨房套装，或者分步骤教学：1.抱着孩子开水龙头（输入\"开大水\"）\"\n[49]     \"晚餐时包子的温度则是从冰箱里取出的。她问孩子，是喜欢冷的包子还是热的包子。通过这种方式，母亲明白了孩子的偏好，即喜欢温暖的包子。这种通过选择疑问句，或是像我之前提到的喝水与不同水温的案例，都能实现高效的输入。\n如果进行测试，也可以通过指令来实现，比如像一般的饮水机，通常有热水龙头和冷水龙头。我们可以让孩子去开水龙头，选择热水或冷水，或者将一杯热水放在那里，让他去拿冷水。再比如，家中的水龙头往左转是热水，往右转是冷水，看看他是否能根据指令正确操作。\n这里需要注意一些细节，即在我们下指令时，比如让孩子去接冷水或拿杯冷水，我们一定要有后续的引导。我们一直强调这个问题，给孩子下的任何指令都必须明确其目的和时机，不能含糊其辞。为什么要这样做？什么时候需要这样做？这些都是我们需要明确的问题。\n还有，就是那个什么时候拿热水壶泡茶拿着吃？那么如果孩子答错了，因为孩子本身没有生活体验，比如冷水泡茶，那么你可以做给他看，用冷水会是什么结果？我们要教孩子一个概念，一定要把这个概念融入到孩子的生活中，让孩子亲身体验，理解对错。\n其实，说穿了就是输入和测试，实际上就是一般小孩子最容易学的两个词汇，一个是“烫”，一个是“辣”，都是带来强烈刺激的感觉，基本上一两次他就明白了，而且有时候还想吃的东西好热的，太烫了。还有一个就是关于气体的词，实际上就是气温。那我们怎么操作呢？现在有空调比较方便，一个房间开空调，一个房间不开，制造出温差，或者利用汽车内的空调。然后当孩子从一个空间切换到另一个空间时，就能感受到温差。还有一些直观的例子，\"\n[50]     \"。也是输入和测试两个阶段，输入的话要以生活体验为底层目标。咱们来打个比方，不只是喝水，比如说到水，你洗衣服的时候洗衣机里，多放点水，或者少放点水，关键是要知道什么时候该多放，什么时候该少放，浇花的时候可能就得少放一些。食物方面就更不用说了，这跟孩子的生活紧密相关，记住我之前的话，听懂了就行。记住，金字塔模型的下面几层是我们要负责的，上面两层是孩子自己的事情。大家自己看看，我就不一一读出来了。选择疑问句可以用来输入输出。就像在形容词测试中，我们经常用到区辨复合指令。\n我们后面形容词汇讲的比较快，然后我们会放一些录像给大家看，多少大家还有疑问嘛？没疑问，我们就直接往后面讲了。\n干湿的话其实无非也是一个对比，但是如果我们按照金字塔来推的话，先给他一些事件，关于干湿的事件，比如说他的衣服湿了，他这个经历有没有过？给他换上干的了，这个经历有没有过？而且我们再想一想，什么东西只有干了才能用，什么东西只有湿了才能用？有没有让他穿过湿的衣服让他有种不适感？这些让他先经历过，然后就是输入和测试。输入的时候手心出汗，就像是沾了水一样。或者拖把弄湿了，还有尿湿了裤子，我们得换条干的裤子，然后顺便洗洗手。把这个小手弄湿，我们才能打肥皂。如果你想让他体验这种感觉，你也可以试试，手干的时候就给他擦肥皂，擦几下发现不行，然后用水把手弄湿，这时候肥皂就能抹上去了。\n这种其实都是很好的样本，但是我们家长最容易犯的错误就是我们家长的习惯太强大了，我们从来可能都不会想到拿干的手去擦肥皂这件事情。但实际上体验过孩子就有印象。那么最简单，\"\n\n问题:\n洗澡的时候，冷热水一次性输入，还是先开热水再叫他开冷水"}],
  temperature=0.3,
#   top_p=0.7,
  max_tokens=4096,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")



<think>
好的，用户的问题是：“洗澡的时候，冷热水一次性输入，还是先开热水再叫他开冷水”。我需要根据提供的已知信息来给出专业且简洁的回答。

首先，我需要回顾已知内容中关于冷热输入的指导。根据内容，源爸多次强调使用对比法，尤其是空间和时间上的对比。例如，同时呈现冷热物品（如冷热包子）或通过时间变化（如加热或冷却）来输入概念。同时，要注意避免混淆，一次只输入一个概念，确保孩子明确感受到冷或热。

在洗澡场景中，源爸提到可以通过突然改变水温制造反差，比如平时用热水突然换成冷水，这时输入“水太冷了”。但要注意不能频繁使用，以免孩子产生抗拒。测试时，可以通过指令让孩子操作水龙头，比如往左是热水，往右是冷水，观察是否能正确执行。

用户的问题是关于在洗澡时如何输入冷热概念，是一次性同时输入还是分阶段。根据已知信息，源爸建议在初期使用极端的对比样本，分开输入冷和热，避免同时提及。例如，在洗澡时先稳定使用热水，然后突然调成冷水，输入“冷”，之后再换回热水，输入“热”。这样分阶段的反差体验能加深理解，同时避免混淆。

此外，测试时要确保孩子真正理解，比如通过指令让孩子选择冷热水龙头。输入时应确保孩子实际感受到温度差异，并注意不要在同一句话中混合冷热，例如不说“水太冷换热水”，而是分开处理。

总结：应该分阶段输入，先制造明显温差体验，分别输入冷和热，确保孩子明确感知后再进行测试。避免同时输入导致混淆。
</think>

根据源爸的教学建议，冷热水输入应分阶段进行，避免一次性混合输入造成混淆：

**初期教学步骤：**
1. **单次聚焦原则**：洗澡时先稳定使用热水，输入“热水/热热的”；随后突然调至冷水制造强烈反差，立刻输入“冷水/好冷啊”。每次只强调单一温度概念，确保孩子明确感知温差。

**进阶操作：**
2. **时间对比法**：待孩子初步理解后，可引导观察水温变化。例如加热冷水时输入“水变热了”，或关闭热水后输入“水慢慢冷了”，通过动态过程强化认知。

**测试与巩固：**
3. **指令测试**：通过水龙头操作验证理解（左热右冷）。例如指令“开热水”时观察是否转向左侧，测试前需确保孩子已多次接触冷热水的触觉差异。

**注意事项：**
- 避免同一句话混合冷热描述（如不说“水太冷换热水”），防止概念混淆。
- 反差体验需适度，频繁切换水温易引发抗拒情绪，影响日常